<a href="https://www.arduino.cc/"><img src="https://raw.githubusercontent.com/sandeepmistry/aimldevfest-workshop-2019/master/images/Arduino_logo_R_highquality.png" width=200/></a>
# Tiny ML en Arduino
## Tutorial de reconocimiento de gestor
 * Sandeep Mistry - Arduino
 * Don Coleman - Chariot Solutions

Traducción a castellano por César García Saéz de La Hora Maker (http://www.lahoramaker.com) 
 
Repositorio original: https://github.com/arduino/ArduinoTensorFlowLiteTutorials/

Repositorio traducido: https://github.com/elsatch/TutorialesArduinoTensorFlowLite/

## Configurar el entorno Python

Esta celda configura las dependencias requeridas para el cuaderno. Ejecútala pulsando el botón a la izquierda.

In [0]:
# Configurar el entorno
!apt-get -qq install xxd
!pip install pandas numpy matplotlib
!pip install tensorflow==2.0.0-rc1

# Cargar los datos

1. Abre el panel de la izquierda del Colab pulsando sobre la flecha  __>__
2. Selecciona la pestaña Ficheros (files)
3. Arrastra los ficheros `saludo.csv` y `pulgarabajo.csv` desde tu ordenador para subirlos dentro de tu proyecto de Colab.

# Mostrar gráficamente los datos (opcional)

Vamos a dibujar los datos de los ficheros de entrada en dos gráficos separados, aceleración y giróscopo, ya que cada conjunto de datos tiene distintas unidades y escalas.

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

filename = "saludo.csv"

df = pd.read_csv("/content/" + filename)

index = range(1, len(df['aX']) + 1)

plt.rcParams["figure.figsize"] = (20,10)

plt.plot(index, df['aX'], 'g.', label='x', linestyle='solid', marker=',')
plt.plot(index, df['aY'], 'b.', label='y', linestyle='solid', marker=',')
plt.plot(index, df['aZ'], 'r.', label='z', linestyle='solid', marker=',')
plt.title("Acceleration")
plt.xlabel("Sample #")
plt.ylabel("Acceleration (G)")
plt.legend()
plt.show()

plt.plot(index, df['gX'], 'g.', label='x', linestyle='solid', marker=',')
plt.plot(index, df['gY'], 'b.', label='y', linestyle='solid', marker=',')
plt.plot(index, df['gZ'], 'r.', label='z', linestyle='solid', marker=',')
plt.title("Gyroscope")
plt.xlabel("Sample #")
plt.ylabel("Gyroscope (deg/sec)")
plt.legend()
plt.show()


# Entrenar la red neuronal





## Interpretar y preparar los datos

La siguiente celda interpreta los contenidos de los ficheros csv y los transforma a un formato que pueder ser utilizado para entrenar la red neuronal totalmente conectada.

Actualiza la lista `GESTURES` con los datos de los distintos gestos que has recogido en formato `.csv`.


In [0]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

print(f"TensorFlow version = {tf.__version__}\n")

# Set a fixed random seed value, for reproducibility, this will allow us to get
# the same random numbers each time the notebook is run
SEED = 1337
np.random.seed(SEED)
tf.random.set_seed(SEED)

# the list of gestures that data is available for
GESTURES = [
    "saludo",
    "pulgarabajo",
]

SAMPLES_PER_GESTURE = 119

NUM_GESTURES = len(GESTURES)

# create a one-hot encoded matrix that is used in the output
ONE_HOT_ENCODED_GESTURES = np.eye(NUM_GESTURES)

inputs = []
outputs = []

# read each csv file and push an input and output
for gesture_index in range(NUM_GESTURES):
  gesture = GESTURES[gesture_index]
  print(f"Processing index {gesture_index} for gesture '{gesture}'.")
  
  output = ONE_HOT_ENCODED_GESTURES[gesture_index]
  
  df = pd.read_csv("/content/" + gesture + ".csv")
  
  # calculate the number of gesture recordings in the file
  num_recordings = int(df.shape[0] / SAMPLES_PER_GESTURE)
  
  print(f"\tThere are {num_recordings} recordings of the {gesture} gesture.")
  
  for i in range(num_recordings):
    tensor = []
    for j in range(SAMPLES_PER_GESTURE):
      index = i * SAMPLES_PER_GESTURE + j
      # normalize the input data, between 0 to 1:
      # - acceleration is between: -4 to +4
      # - gyroscope is between: -2000 to +2000
      tensor += [
          (df['aX'][index] + 4) / 8,
          (df['aY'][index] + 4) / 8,
          (df['aZ'][index] + 4) / 8,
          (df['gX'][index] + 2000) / 4000,
          (df['gY'][index] + 2000) / 4000,
          (df['gZ'][index] + 2000) / 4000
      ]

    inputs.append(tensor)
    outputs.append(output)

# convert the list to numpy array
inputs = np.array(inputs)
outputs = np.array(outputs)

print("Data set parsing and preparation complete.")

## Aleatorizar y dividir las entradas y salidas para el entrenamiento

Dividir las entradas y salidas en tres conjuntos de datos: 60% para entrenamiento, 20% para validación, y 20% para pruebas.

  - El conjunto de entrenamiento se utiliza para entrenar el modelo
  - El conjunto de validación se utilizar para medir lo bien que va entrenándose el modelo durante el entrenamiento
  - El conjunto de pruebas se emplean para verificar el modelo después de entreamiento

In [0]:
# Randomize the order of the inputs, so they can be evenly distributed for training, testing, and validation
# https://stackoverflow.com/a/37710486/2020087
num_inputs = len(inputs)
randomize = np.arange(num_inputs)
np.random.shuffle(randomize)

# Swap the consecutive indexes (0, 1, 2, etc) with the randomized indexes
inputs = inputs[randomize]
outputs = outputs[randomize]

# Split the recordings (group of samples) into three sets: training, testing and validation
TRAIN_SPLIT = int(0.6 * num_inputs)
TEST_SPLIT = int(0.2 * num_inputs + TRAIN_SPLIT)

inputs_train, inputs_test, inputs_validate = np.split(inputs, [TRAIN_SPLIT, TEST_SPLIT])
outputs_train, outputs_test, outputs_validate = np.split(outputs, [TRAIN_SPLIT, TEST_SPLIT])

print("Data set randomization and splitting complete.")

## Construir y entrenar el modelo

Construimos y entrenamos un modelo de [TensorFlow](https://www.tensorflow.org) utilizando la API de alto nivel [Keras](https://www.tensorflow.org/guide/keras).

In [0]:
# build the model and train it
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(50, activation='relu')) # relu is used for performance
model.add(tf.keras.layers.Dense(15, activation='relu'))
model.add(tf.keras.layers.Dense(NUM_GESTURES, activation='softmax')) # softmax is used, because we only expect one gesture to occur per input
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
history = model.fit(inputs_train, outputs_train, epochs=600, batch_size=1, validation_data=(inputs_validate, outputs_validate))



## Verificación

Dibujamos el rendimiento del modelo frente a la validación.


### Dibujar las pérdidas

Dibujamos las pérdidas para detectar cuando deja de mejorar el modelo

In [0]:
# increase the size of the graphs. The default size is (6,4).
plt.rcParams["figure.figsize"] = (20,10)

# graph the loss, the model above is configure to use "mean squared error" as the loss function
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'g.', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

print(plt.rcParams["figure.figsize"])

### Dibujamos las péridas de nuevo, saltando un poco del principio

Vamos a dibujar los mismos gráficos de la celda anterior, pero empezaremos en el índice 100 de forma que podemos ampliar la gráfica una vez que el modelo comience a converger.

In [0]:
# graph the loss again skipping a bit of the start
SKIP = 100
plt.plot(epochs[SKIP:], loss[SKIP:], 'g.', label='Training loss')
plt.plot(epochs[SKIP:], val_loss[SKIP:], 'b.', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

### Dibujamos el error absoluto medio

[Error absoluto medio](https://es.wikipedia.org/wiki/Error_absoluto_medio) es otra métrica para medir el rendimiento del modelo.



In [0]:
# graph of mean absolute error
mae = history.history['mae']
val_mae = history.history['val_mae']
plt.plot(epochs[SKIP:], mae[SKIP:], 'g.', label='Training MAE')
plt.plot(epochs[SKIP:], val_mae[SKIP:], 'b.', label='Validation MAE')
plt.title('Training and validation mean absolute error')
plt.xlabel('Epochs')
plt.ylabel('MAE')
plt.legend()
plt.show()


### Ejecutar con datos de prueba
Añadimos nuestros datos de prueba en el modelo y dibujamos las predicciones


In [0]:
# use the model to predict the test inputs
predictions = model.predict(inputs_test)

# print the predictions and the expected ouputs
print("predictions =\n", np.round(predictions, decimals=3))
print("actual =\n", outputs_test)

# Plot the predictions along with to the test data
plt.clf()
plt.title('Training data predicted vs actual values')
plt.plot(inputs_test, outputs_test, 'b.', label='Actual')
plt.plot(inputs_test, predictions, 'r.', label='Predicted')
plt.show()

# Convertimos el modelo entrenado a Tensor Flow Lite

La celda siguiente convierte el modelo al formato TFlite. También se imprime el tamaño del modelo en bytes.

In [0]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model to disk
open("gesture_model.tflite", "wb").write(tflite_model)
  
import os
basic_model_size = os.path.getsize("gesture_model.tflite")
print("Model is %d bytes" % basic_model_size)
  
  

## Codificar el modelo en un fichero de cabeceras de Arduino

La siguiente celda crea un array de bytes constantes que contiene el modelo TFlite. Lo importaremos como una pestaña con el siguiente programa.

In [0]:
!echo "const unsigned char model[] = {" > /content/model.h
!cat gesture_model.tflite | xxd -i      >> /content/model.h
!echo "};"                              >> /content/model.h

import os
model_h_size = os.path.getsize("model.h")
print(f"Header file, model.h, is {model_h_size:,} bytes.")
print("\nOpen the side panel (refresh if needed). Double click model.h to download the file.")

# Clasificando los datos de la IMU

Es hora de volver a las instrucciones del tutorial y ejecutar nuestro nuevo modelo en el Arduino Nano 33 BLE Sense para clasificar los datos del giróscopo y acelerómetro.
